<a href="https://colab.research.google.com/github/bluecityisu/class2022/blob/main/LSTM_RandForest_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [103]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [104]:
def create_dataset(dataset, look_back):
 dataX, dataY = [], []
 for i in range(len(dataset)-look_back):
  a = dataset.iloc[i:(i+look_back), 1:]
  dataX.append(a)
  dataY.append(dataset.iloc[i + look_back, 0])
 return np.array(dataX), np.array(dataY)

In [105]:
url='/content/drive/MyDrive/專題2024/lstm/data.xlsx'
data = pd.read_excel(url)

data.tail()

,t,Y9999,Y99992,2302 麗正,2303 聯電,2329 華泰
66,2023-05-12,-0.007917,0,-0.017588,-0.018145,-0.040816
67,2023-05-19,0.043384,1,-0.023018,0.026694,0.021277
68,2023-05-26,0.020410,1,0.013089,0.008000,0.031250
69,2023-06-02,0.012230,1,0.082687,0.019841,0.065657
70,2023-06-09,0.010743,1,0.026253,0.017510,0.090047


## **chapter 2 隨機森林**

In [106]:
data2=data.drop(['t'], axis=1)
data2=data2.drop(['Y9999'], axis=1)
data2.iloc[:,0]

0     0
1     0
2     1
3     0
4     0
     ..
66    0
67    1
68    1
69    1
70    1
Name: Y99992, Length: 71, dtype: int64

In [124]:
look_back = 1
x, y = create_dataset(data2, look_back)
x=np.reshape(x, (x.shape[0],x.shape[2]))
x.shape

(70, 3)

In [125]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
clfr = RandomForestClassifier(max_depth=3, n_jobs=-1)
clfr.fit(x, y)   #開始計算
y_pred = clfr.predict(x)
#print(classification_report(y, y_pred))

In [129]:
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
# 建立特徵選取器，門檻值預設為重要性的平均值
selectorr = SelectFromModel(clfr)
selectorr.fit(x, y)
print('門檻值 =', selectorr.threshold_)
print('特徵遮罩：', selectorr.get_support())

# 選出新特徵，重新訓練隨機森林
x_new = selectorr.transform(x)
x_new=pd.DataFrame(x_new)
y=pd.DataFrame(y)

from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42).split(x, y)
score_lst = []  # 紀錄 k 次交叉驗證的正確率

for k, (i_train, i_valid) in enumerate(kfold):
  clfr.fit(x_new.iloc[i_train, :], y.iloc[i_train])

  y_pred_test = clfr.predict(x_new.iloc[i_valid, :])

  score=accuracy_score(y.iloc[i_valid], y_pred_test)
  print('precision=',score)
  score_lst.append(score)
  print('%2d-Fold: Acc=%.3f' % (k+1, score))

print('\n10-fold CV accuracy = %.3f, std = %.3f' % (np.mean(score_lst), np.std(score_lst)))
print(np.mean(score_lst)/np.std(score_lst))

門檻值 = 0.3333333333333333
特徵遮罩： [ True  True False]


<ipython-input-129-e98e52514f32>:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clfr.fit(x_new.iloc[i_train, :], y.iloc[i_train])


precision= 0.2857142857142857
 1-Fold: Acc=0.286
precision= 0.7142857142857143
 2-Fold: Acc=0.714


<ipython-input-129-e98e52514f32>:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clfr.fit(x_new.iloc[i_train, :], y.iloc[i_train])
<ipython-input-129-e98e52514f32>:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clfr.fit(x_new.iloc[i_train, :], y.iloc[i_train])


precision= 0.42857142857142855
 3-Fold: Acc=0.429


...

precision= 0.5714285714285714
 8-Fold: Acc=0.571


<ipython-input-129-e98e52514f32>:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clfr.fit(x_new.iloc[i_train, :], y.iloc[i_train])


precision= 0.14285714285714285
 9-Fold: Acc=0.143


<ipython-input-129-e98e52514f32>:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clfr.fit(x_new.iloc[i_train, :], y.iloc[i_train])


precision= 0.8571428571428571
10-Fold: Acc=0.857

10-fold CV accuracy = 0.486, std = 0.194
2.506513254633251


# **chapter 3 LSTM**

In [158]:
data2=data.drop(['t'], axis=1)
data2=data2.drop(['Y9999'], axis=1)
y=data2.pop("Y99992")
x = selectorr.transform(data2)
x=pd.DataFrame(x)
data2=pd.concat([y,x], axis=1)
data2

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


,Y99992,0,1
0,0,-0.018868,0.001595
1,0,0.000000,-0.087580
2,1,0.064103,-0.033159
3,0,-0.036145,-0.030686
4,0,-0.034375,-0.029795
...,...,...,...
66,0,-0.017588,-0.018145
67,1,-0.023018,0.026694
68,1,0.013089,0.008000
69,1,0.082687,0.019841


In [159]:
train_size = int(len(data2) * 0.67)
test_size = len(data2) - train_size
train, test = data2.iloc[0:train_size,:], data2.iloc[train_size:len(data2),:]
train.tail()

,Y99992,0,1
42,1,0.002674,0.036364
43,1,-0.013333,0.004386
44,1,0.021622,-0.006550
45,0,-0.063492,-0.043956
46,0,0.008475,0.018391


In [160]:
look_back = 5
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
trainY

array([1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0])

In [161]:
from keras.utils import to_categorical
trainY = to_categorical(trainY, dtype ="uint8")
testY = to_categorical(testY, dtype ="uint8")


In [162]:
trainY.shape

(42, 2)

In [163]:
model = Sequential([
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(4,input_shape=[look_back, trainX.shape[2]],return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(8)),
  Dense(128, activation='relu'),
  Dense(64, activation='relu'),
  Dense(32, activation='relu'),
  Dense(2, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [164]:
model.fit(trainX, trainY, epochs=1000, verbose=2)

Epoch 1/1000
2/2 - 9s - loss: 0.6933 - accuracy: 0.5000 - 9s/epoch - 4s/step
Epoch 2/1000
2/2 - 0s - loss: 0.6925 - accuracy: 0.5476 - 22ms/epoch - 11ms/step
Epoch 3/1000
2/2 - 0s - loss: 0.6922 - accuracy: 0.5238 - 22ms/epoch - 11ms/step
Epoch 4/1000
2/2 - 0s - loss: 0.6919 - accuracy: 0.5476 - 21ms/epoch - 10ms/step
Epoch 5/1000
2/2 - 0s - loss: 0.6914 - accuracy: 0.4762 - 20ms/epoch - 10ms/step
Epoch 6/1000
2/2 - 0s - loss: 0.6910 - accuracy: 0.5238 - 21ms/epoch - 10ms/step
Epoch 7/1000
2/2 - 0s - loss: 0.6911 - accuracy: 0.5238 - 20ms/epoch - 10ms/step
Epoch 8/1000
2/2 - 0s - loss: 0.6905 - accuracy: 0.5238 - 20ms/epoch - 10ms/step
Epoch 9/1000
2/2 - 0s - loss: 0.6903 - accuracy: 0.5238 - 21ms/epoch - 10ms/step
Epoch 10/1000
2/2 - 0s - loss: 0.6898 - accuracy: 0.5238 - 21ms/epoch - 11ms/step
Epoch 11/1000
2/2 - 0s - loss: 0.6898 - accuracy: 0.5238 - 24ms/epoch - 12ms/step
Epoch 12/1000
2/2 - 0s - loss: 0.6890 - accuracy: 0.5238 - 22ms/epoch - 11ms/step
Epoch 13/1000
2/2 - 0s - loss

In [165]:
testPredict = model.predict(testX)
#testPredict=np.reshape(testPredict, -1)
print(testY)
print(testPredict)

1/1 [==============================] - 1s 1s/step
[[0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]
[[5.27449957e-15 1.00000000e+00]
 [6.87611474e-32 1.00000000e+00]
 [4.10810595e-26 1.00000000e+00]
 [1.42900281e-20 1.00000000e+00]
 [9.99997854e-01 2.11761903e-06]
 [1.00000000e+00 4.81365842e-14]
 [1.00000000e+00 5.05563313e-10]
 [4.29261178e-02 9.57073867e-01]
 [9.99999762e-01 2.62225115e-07]
 [1.00000000e+00 1.47402040e-21]
 [1.00000000e+00 1.37731925e-26]
 [1.00000000e+00 1.95204909e-13]
 [1.00000000e+00 9.30372519e-16]
 [3.65826488e-03 9.96341765e-01]
 [9.99915004e-01 8.50028518e-05]
 [8.75916719e-01 1.24083266e-01]
 [3.27032612e-04 9.99673009e-01]
 [4.97257803e-03 9.95027483e-01]
 [4.44212917e-13 1.00000000e+00]]


In [166]:
def deter(x):
  de=[]
  for i in range(len(x)):
    if x[i][0]>x[i][1]:
      de.append(0)
    else:
      de.append(1)
  return de

In [167]:
testY1=deter(testY)
testPredict1=deter(testPredict)
print(testY1)
print(testPredict1)

[1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1]
[1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1]


In [168]:
from sklearn.metrics import accuracy_score
score=accuracy_score(testY1, testPredict1)
score

0.7368421052631579

In [169]:
test.iloc[:,1:]

,0,1
47,-0.044818,-0.053047
48,-0.020528,-0.029797
49,0.005988,0.041769
50,-0.071429,0.070755
51,0.012821,0.012115
52,0.069620,0.090316
53,0.011834,0.013972
54,0.029240,-0.015748
55,-0.005682,-0.001000
56,0.002857,0.027027


In [170]:
recent=test.iloc[test.shape[0]-look_back:test.shape[0], 1:]
recent=np.reshape(recent, (1, look_back, test.shape[1]-1))
recent

array([[[-0.01758794, -0.01814516],
        [-0.0230179 ,  0.02669405],
        [ 0.01308901,  0.008     ],
        [ 0.08268734,  0.01984127],
        [ 0.02625298,  0.01750973]]])

In [171]:
recentP = model.predict(recent)
print(deter(recentP))


1/1 [==============================] - 0s 20ms/step
[1]
